In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs4
import pandas as pd
import lxml
import time

In [ ]:
years = list(range(2023,2020,-1))
all_seasons = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats";
for year in years:
    data = requests.get(standings_url)
    soup = bs4(data.text)
    standings_table = soup.select_one("table.stats_table")
    st_links = standings_table.find_all("a")
    st_hrefs = [link.get("href") for link in st_links]
    st_hrefs =[l for l in st_hrefs if '/squads/' in l]
    full_links = [f"https://fbref.com{l}" for l in st_hrefs]
    previous_season = soup.select_one("a.prev").get("href")
    standings_url = f"https://fbref.com{previous_season}"
    for team_url in full_links:
        team_name = team_url.split('/')[-1].replace('-Stats', '').replace('-',' ')
        team_stats = requests.get(team_url)
        team_soup = bs4(team_stats.text)
        matches_tb = pd.read_html(team_stats.text, match="Scores & Fixtures")

        all_links = team_soup.find_all('a')

        all_links = [l.get("href") for l in all_links]

        shoot_link = [l for l in all_links if l and 'all_comps/shooting/' in l]
        matches_tb=matches_tb[0]
        shoot_url = requests.get(f"https://fbref.com{shoot_link[0]}");
        shoot_tb = pd.read_html(shoot_url.text, match="Shooting")
        shoot_tb[0].columns = shoot_tb[0].columns.droplevel()
        shoot_tb=shoot_tb[0]
        try:
            team_data = matches_tb.merge(shoot_tb[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        
        team_data['Seasons'] = year
        team_data['Team'] = team_name
        all_seasons.append(team_data)
        time.sleep(1)